In [99]:
import pandas as pd

data = pd.read_csv("dataset/dataset2.csv")
# print("Num Samples:", data.shape[0])

# NOTE: Remove this line before final training
data = data[0:20000]
print("Num Samples:", data.shape[0])

Num Samples: 20000


In [100]:
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Data Cleanup

In [101]:
import re

def clean(text):
    text = str(text).lower()
    text = text.replace("\n", " ")
    text = re.sub("[^a-z A-Z]+", "", text) # only keep digits and spaces
    text = re.sub('\s+', " ", text) # clean up spaces
    return text.strip()
        
data["comment_text"] = data["comment_text"].apply(clean)

In [102]:
data.head()

,comment_text,toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
3,more i cant make any real suggestions on impro...,0
4,you sir are my hero any chance you remember wh...,0


In [103]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maharsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words=stopwords,
    strip_accents='unicode', 
    ngram_range=(1,3),
    max_features=10000,
)

X = vectorizer.fit_transform(data["comment_text"])
y = data.drop(labels=["comment_text"], axis=1)

In [106]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16000, 10000) (16000, 1)
(4000, 10000) (4000, 1)


In [107]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
y_predicted = classifier.predict(X_test)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [108]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

report = classification_report(y_test, y_predicted)
print(report)
print(confusion_matrix(y_test, y_predicted))

print("accuracy:")
print(accuracy_score(y_test, y_predicted))

print("f1 score:")
print(f1_score(y_test, y_predicted))

print("precision:")
print(precision_score(y_test, y_predicted))

print("recall:")
print(recall_score(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3614
           1       0.98      0.43      0.60       386

    accuracy                           0.94      4000
   macro avg       0.96      0.72      0.78      4000
weighted avg       0.95      0.94      0.93      4000

[[3610    4]
 [ 219  167]]
accuracy:
0.94425
f1 score:
0.599640933572711
precision:
0.9766081871345029
recall:
0.4326424870466321
